In [4]:
import warnings
warnings.filterwarnings('ignore')

import requests
import geopandas as gpd
import pandas as pd
import json, time, tqdm

api_key = "google_maps_api_goes_here"
payload={}
headers = {}

df = pd.DataFrame()

In [3]:
input_grid = gpd.read_file('./input_grid/grid.shp')
input_grid.to_crs(epsg=4326, inplace=True)
input_grid.head()

,id,left,top,right,bottom,geometry
0,1.0,479865.279675,4.294371e+06,480365.279675,4.294871e+06,POINT (-9.23187 38.79786)
1,2.0,479865.279675,4.293871e+06,480365.279675,4.294371e+06,POINT (-9.23185 38.79335)
2,3.0,479865.279675,4.293371e+06,480365.279675,4.293871e+06,POINT (-9.23184 38.78885)
3,4.0,479865.279675,4.292871e+06,480365.279675,4.293371e+06,POINT (-9.23182 38.78434)
4,5.0,479865.279675,4.292371e+06,480365.279675,4.292871e+06,POINT (-9.23181 38.77984)


In [ ]:
place_id_duplicates_check = {}
skipped = 0

for point in tqdm.tqdm(input_grid['geometry']):
    
    lat = str(point.y)
    lng = str(point.x)
    
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?location="+lat+"%2C"+lng+"&radius=250&key="+api_key
    ## restricted to 250 meter readius
    
    while True:

        response = requests.request("GET", url, headers=headers, data=payload)
        poi = json.loads(response.text)

        time.sleep(5)

        for i in range(len(poi['results'])):
            
            place_id = poi['results'][i]['place_id']
            
            if place_id in place_id_duplicates_check.keys():
                skipped += 1
                continue
            
            place_id_duplicates_check[place_id] = 1
            
            place_lat = poi['results'][i]['geometry']['location']['lat']
            place_lng = poi['results'][i]['geometry']['location']['lng']
            name = poi['results'][i]['name']

            try:
                rating = poi['results'][i]['rating']
            except:
                rating = -99

            category = poi['results'][i]['types']

            try:
                user_ratings_total = poi['results'][i]['user_ratings_total']
            except:
                user_ratings_total = -99
            
            try:
                vicinity = poi['results'][i]['vicinity']
            except:
                vicinity = "N/A"

            df = df.append([[place_lat, place_lng, name, rating, category, user_ratings_total, vicinity, place_id]])

        if 'next_page_token' not in poi.keys():
            break
        else:
            pagetoken = poi['next_page_token']
            url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?location="+lat+"%2C"+lng+"&radius=250&key="+api_key+"&pagetoken="+pagetoken
    
df.columns = ['place_lat','place_lng','name','rating','category','user_ratings_total','vicinity','place_id']
df.shape

In [ ]:
df.drop_duplicates(subset='place_id', keep='first', inplace=True)
df.to_csv('./backup_pois.csv', index=False)

In [ ]:
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.place_lng, df.place_lat))
gdf.head()

In [ ]:
gdf['category'] = gdf['category'].apply(lambda x: ",".join(x))

In [ ]:
gdf.set_crs(epsg=4326, inplace=True)
gdf.to_file('./google_poi.shp', driver="ESRI Shapefile")

In [ ]:
## Get ratings of POIs

reviews = pd.DataFrame()
failed = []

for i in tqdm.tqdm(range(df.shape[0])):
    
    try:
        if df['user_ratings_total'].iloc[i] != -99:
            place_id = df['place_id'].iloc[i]
        else:
            continue

        review_url = "https://maps.googleapis.com/maps/api/place/details/json?place_id="+place_id+"&fields=review&key="+api_key

        payload={}
        headers = {}

        review_response = requests.request("GET", review_url, headers=headers, data=payload)
        review_response_json = json.loads(review_response.text)
        time.sleep(1)

        for review in review_response_json['result']['reviews']:
            review['place_id'] = place_id
            reviews = reviews.append([review])
    except:
        failed.append(i)
        continue

reviews.shape

In [ ]:
reviews.to_csv('./pois_reviews.csv', index=False)